In [ ]:
!pip install transformers datasets evaluate sentencepiece

In [ ]:
#T5 SMALL HAS 60 MILLION PARAMETERS AND T5 BASE HAS 220 MILLION PARAMETERS

In [ ]:
from datasets import load_dataset

# From hugging face import samsum dataset has 14k samples of convos
dataset = load_dataset("knkarthick/samsum")
print("Dataset loaded:")
print(dataset)


print("\n--- Sample ---")
print("DIALOGUE:")
print(dataset["train"][0]["dialogue"])
print("\nSUMMARY:")
print(dataset["train"][0]["summary"])

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14731 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Dataset loaded:
DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

--- Sample ---
DIALOGUE:
Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)

SUMMARY:
Amanda baked cookies and will bring Jerry some tomorrow.


In [ ]:
from transformers import AutoTokenizer

# Load the T5-small tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")
prefix = "summarize: "

# T5's max length is 512
max_input_length = 512
max_target_length = 128

def preprocess_function_samsum(examples):
    # Prepare the inputs (the 'dialogue')
    inputs = [prefix + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    # Prepare the "labels" (the 'summary')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing
# Runs on all 14,732 train samples
tokenized_dataset = dataset.map(preprocess_function_samsum, batched=True)

print("\nTokenizing complete.")
print(tokenized_dataset)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]


Tokenizing complete.
DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
})


In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load the T5-small model
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-samsum-results",    # Directory to save the model
    eval_strategy="epoch",         # Evaluate at the end of each epoch
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    report_to="none",                    # or uses wandib to show history
    num_train_epochs=8,                  # We'll train for 3 epochs
    predict_with_generate=True,          # Required for summarization metrics
    fp16=True,                           # Use 16-bit precision
)

# The data collator dynamically pads sequences to the max length in a batch
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Create the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-3097422875.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# Start the training!
print("Starting T5 training on SAMSum...")
trainer.train()
print("Training finished.")

# Save the final model & tokenizer
trainer.save_model("./t5-samsum-model")

Starting T5 training on SAMSum...


Epoch,Training Loss,Validation Loss
1,0.428700,0.382812
2,0.392900,0.371210
3,0.355300,0.369359


Training finished.


In [ ]:
from transformers import pipeline

# Load the summarization pipeline with our fine-tuned model
summarizer_pipe = pipeline("summarization", model="./t5-samsum-model", tokenizer=tokenizer)

# Grab a dialogue from the test set
test_dialogue = dataset["test"][10]["dialogue"]

print("\n--- Testing with a new dialogue ---")
print("\nORIGINAL DIALOGUE:")
print(test_dialogue)

# Generate summary
summary = summarizer_pipe(test_dialogue, max_length=100, min_length=10, do_sample=False)

print("\nGENERATED SUMMARY:")
print(summary[0]['summary_text'])

print("\nACTUAL HUMAN SUMMARY:")
print(dataset["test"][10]["summary"])

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Testing with a new dialogue ---

ORIGINAL DIALOGUE:
Wanda: Let's make a party!
Gina: Why?
Wanda: beacuse. I want some fun!
Gina: ok, what do u need?
Wanda: 1st I need too make a list
Gina: noted and then?
Wanda: well, could u take yours father car and go do groceries with me?
Gina: don't know if he'll agree
Wanda: I know, but u can ask :)
Gina: I'll try but theres no promisess
Wanda: I know, u r the best!
Gina: When u wanna go
Wanda: Friday?
Gina: ok, I'll ask

GENERATED SUMMARY:
Gina and Wanda will go to a party on Friday.

ACTUAL HUMAN SUMMARY:
Wanda wants to throw a party. She asks Gina to borrow her father's car and go do groceries together. They set the date for Friday. 


In [ ]:
# TEST IT WITH T5 small without finetunig and ask the same question

In [1]:
from transformers import pipeline, AutoTokenizer

# Load the ORIGINAL t5-small, NOT your finetuned one
tokenizer = AutoTokenizer.from_pretrained("t5-small")
summarizer = pipeline("summarization", model="t5-small", tokenizer=tokenizer)

# Get a dialogue from the test set
test_dialogue = """
Wanda: Let's make a party!
Gina: Why?
Wanda: beacuse. I want some fun!
Gina: ok, what do u need?
Wanda: 1st I need too make a list
Gina: noted and then?
Wanda: well, could u take yours father car and go do groceries with me?
Gina: don't know if he'll agree
Wanda: I know, but u can ask :)
Gina: I'll try but theres no promisess
Wanda: I know, u r the best!
Gina: When u wanna go
Wanda: Friday?
Gina: ok, I'll ask
"""

# Ask the original model to summarize it
print(summarizer(test_dialogue))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
Your max_length is set to 200, but your input_length is only 162. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=81)


[{'summary_text': "Gina: ok, what do u need? Wanda: I know, u r the best! u'll ask if u wanna go Wanda ."}]
